In [1]:
from numba import cuda
from numba import float32,int32,int64
import numpy as np
import pdb
from math import floor
from decimal import *
a = np.array([2, 1, 6,2, 1, 6] ,dtype = np.int32)

In [3]:
def EGCD(a,b):
    cc = a
    dd = b
    c1 = 1
    c2 =0
    d1 =0
    d2 = 1
    
    while not dd==0:
        try:
            q = cc/dd
        except TypeError:
            pdb.set_trace()
            
        r = cc - int(q*dd)
        r = int(r)
        cc = dd
        dd = r

        r1 = c1 - int(q*d1)
        r1 = int(r1)
        c1 = d1
        d1 = r1

        r2 = c2 - int(q*d2)
        r2 = int(r2)
        c2 = d2
        d2 = r2
    
    return (cc,c1,c2)


In [2]:
@cuda.jit('float32[:](float32[:])',target='device')
def test_array_op(aa):
    for a in aa:
        a*=2
    return a

TypeError: CUDA kernel must have void return type.

In [ ]:
numBlocks = 2
n = len(a)
blockdim = n//numBlocks
griddim = numBlocks
@cuda.jit(argtypes=[float32[:], float32[:],float32[:]], target='gpu')
def CRA_kernel(a,b,c):
    sa = cuda.shared.array(shape=(256,), dtype=int32) # residue array
    sc = cuda.shared.array(shape=(256,),dtype=int32) # precomputed coefficient array
    sM = cuda.shared.array(shape=(256,),dtype=int32) # M coefficient array
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    bw = cuda.blockDim.x
    k = bw
    i = cuda.grid(1)
    if i < len(a):
        sa[tx] = a[i]
        sc[tx] = c[tx]
        sM[tx] = 1
        if tx==0:
            # start from the first thread in the block
            
            for i in range(1,k):
                gamma[i] = ((rvect[i] - gamma[0])*c[i]) % mvect[i]
                M[i] = (mvect[0]*c[i]) % mvect[i]

            for i in range(1,k-1):
                for j in range(i+1,k):
                    gamma[j] = (gamma[j] - gamma[i]*M[j]) % mvect[j]
                        M[j] = (M[j]*mvect[i]) % mvect[j]
        
            b[i] = 
                
            
        

In [ ]:
@cuda.jit('int32[:](int32[:])',device=True, inline = True)
def test_array_op(a):
    for i in range(len(a)):
        a[i] += i
    return a

@cuda.jit(argtypes=[int32[:],int32[:]], target='gpu')
def CRA_kernel(a,res):
    sa = cuda.shared.array(shape=(256,), dtype=int32)
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    bw = cuda.blockDim.x
    
    i = cuda.grid(1)
    if i < len(a):
        sa[tx] = a[i]
        cuda.syncthreads()
        #sc[tx] = c[tx]
        #sM[tx] = 1
        if tx==0:
            os = 3
            sa[tx:os] = test_array_op(sa[tx:os])
            
            res[i:i+os] = sa[tx:os]
            
griddim = 2
blockdim = 3

a = np.array([2, 1, 6,2, 2, 6],dtype=np.int32 )
b = np.empty_like(a)
CRA_kernel[griddim, blockdim](a,b)   
    

In [9]:
@cuda.jit('int32[:](int32[:],int32[:],int32[:],int32[:],int32[:])',device=True, inline = True)
def MRC_alg(gamma,rvect,mvect,c,M):
    # initalizations
    k = len(rvect)
    gamma[0] = rvect[0]
    M[0] = 1
 
    for i in range(1,k):
        gamma[i] = ((rvect[i] - gamma[0])*c[i]) % mvect[i]
        M[i] = (mvect[0]*c[i]) % mvect[i]

    for i in range(1,k-1):
        for j in range(i+1,k):
            gamma[j] = (gamma[j] - gamma[i]*M[j]) % mvect[j]
            M[j] = (M[j]*mvect[i]) % mvect[j]
            
    return gamma

@cuda.jit(argtypes=[int32[:],int32[:],int32[:],int32[:]], target='gpu')
def CRA_kernel(a,res,m,c):
    sa = cuda.shared.array(shape=(256,), dtype=int32) # residues
    sb = cuda.shared.array(shape=(256,), dtype=int32) # gamma Mixed radix vector (returned from the MRC device function)
    sM = cuda.shared.array(shape=(256,), dtype=int32) # M coefficient vector
    sc = cuda.shared.array(shape=(256,), dtype=int32) # precomputed coefficient 
    sm = cuda.shared.array(shape=(256,), dtype=int32) # prime vector

    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    bw = cuda.blockDim.x
    
    i = cuda.grid(1)
    if i < len(a):
        sa[tx] = a[i]
        sc[tx] = c[tx]
        sm[tx] = m[tx]
        cuda.syncthreads()
        #sc[tx] = c[tx]
        #sM[tx] = 1
        if tx==0:
            os = 3
            sb[tx:os] = MRC_alg(sb[tx:os],sa[tx:os],sm[tx:os],sc[tx:os],sM[tx:os])
            
            res[i:i+os] = sb[tx:os]
            
griddim = 2
blockdim = 3

a = np.array([2, 1, 6,0, 1, 2],dtype=np.int32 )
b = np.empty_like(a)
mvect = [1,3,5]
c = [0] + [EGCD(reduce(lambda x,y:x*y,mvect[0:k+1]),m)[1] for (k,m) in enumerate(mvect[1:])]
CRA_kernel[griddim, blockdim](a,b,mvect,c)   
    

TypingError: Caused By:
Traceback (most recent call last):
  File "/Users/kevm1892/anaconda/lib/python2.7/site-packages/numba/compiler.py", line 201, in run
    res = stage()
  File "/Users/kevm1892/anaconda/lib/python2.7/site-packages/numba/compiler.py", line 415, in stage_nopython_frontend
    self.locals)
  File "/Users/kevm1892/anaconda/lib/python2.7/site-packages/numba/compiler.py", line 710, in type_inference_stage
    infer.propagate()
  File "/Users/kevm1892/anaconda/lib/python2.7/site-packages/numba/typeinfer.py", line 408, in propagate
    self.constrains.propagate(self.context, self.typevars)
  File "/Users/kevm1892/anaconda/lib/python2.7/site-packages/numba/typeinfer.py", line 107, in propagate
    constrain(context, typevars)
  File "/Users/kevm1892/anaconda/lib/python2.7/site-packages/numba/typeinfer.py", line 227, in __call__
    self.resolve(context, typevars, fnty)
  File "/Users/kevm1892/anaconda/lib/python2.7/site-packages/numba/typeinfer.py", line 252, in resolve
  

In [ ]:
@cuda.jit('int32[:](int32[:])',device=True, inline = True)
def test_array_op(a):
    for i in range(len(a)):
        a[i] += i
    return a

@cuda.jit(argtypes=[int32[:],int32[:]], target='gpu')
def CRA_kernel(a,res):
    sa = cuda.shared.array(shape=(256,), dtype=int32)
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    bw = cuda.blockDim.x
    
    i = cuda.grid(1)
    if i < len(a):
        sa[tx] = a[i]
        cuda.syncthreads()
        #sc[tx] = c[tx]
        #sM[tx] = 1
        if tx==0:
            os = 3
            sa[tx:os] = test_array_op(sa[tx:os])
            
            res[i:i+os] = sa[tx:os]
            
griddim = 2
blockdim = 3

a = np.array([2, 1, 6,2, 2, 6],dtype=np.int32 )
b = np.empty_like(a)
CRA_kernel[griddim, blockdim](a,b)   
    